# SOLACE	AI

## Problem Statement

#### The primary problem is the unpredictable availability of solar energy during the rainy season, which leads to either inefficient charging or reliance on the traditional power grid. This can cause power fluctuations and increased electricity costs for users. The project's main goal is to create a system that can intelligently manage this variability by optimizing charging schedules and balancing the load between solar power and the grid.

## Abstract

#### This project addresses the challenge of providing a reliable and efficient electric vehicle (EV) charging system that primarily uses solar power, even during periods of low sunlight. Our proposed solution is an AI-powered smart charging load balancer that optimizes energy distribution from solar panels and the grid to EVs. We'll use a machine learning model to predict solar energy output based on weather data and manage charging schedules to prevent grid overload and maximize the use of clean energy. The expected outcomes include a significant increase in the use of renewable energy for EV charging and a reduction in strain on the power grid during peak demand.


## Import Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import joblib

In [2]:
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.20.0-cp313-cp313-win_amd64.whl.metadata (4.6 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.9.23-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached ml_dtypes-0.5.3-cp313-cp313-win_amd64.whl.metadata (9.2 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
  Usi

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\msant\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python313\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\fault_injection\\fault_injection_service_config_parser.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\msant\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### Dataset overview

In [ ]:
# Run this in Google Colab
%pip install pandas numpy scikit-learn matplotlib plotly requests


   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/9.8 MB 11.8 MB/s eta 0:00:01
   -------------- ------------------------- 3.7/9.8 MB 11.7 MB/s eta 0:00:01
   ------------------------- -------------- 6.3/9.8 MB 11.8 MB/s eta 0:00:01
   ----------------------------------- ---- 8.7/9.8 MB 11.8 MB/s eta 0:00:01
   ---------------------------------------- 9.8/9.8 MB 11.3 MB/s  0:00:00

   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   --------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Name : ev_solar_project_dataset
source : koogle.com
file formet : csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import plotly.express as px
import requests
import joblib

In [ ]:
# Simulating EV driving & battery data
np.random.seed(42)

data_size = 500
df = pd.DataFrame({
    'speed_kmh': np.random.randint(20, 120, size=data_size),  # Vehicle speed
    'acceleration': np.random.uniform(0, 3, size=data_size),  # Acceleration m/s^2
    'temperature': np.random.randint(10, 40, size=data_size), # Ambient temperature
    'battery_level': np.random.randint(20, 100, size=data_size), # Battery %
    'distance_km': np.random.uniform(1, 50, size=data_size),  # Trip distance
})

# Target: energy consumed per trip (kWh) - simulated formula
df['energy_consumed'] = (df['distance_km'] * (0.2 + 0.01*df['speed_kmh'])) + (df['acceleration']*0.5)
df
df.head()
# Splitting data
X = df.drop(columns=['energy_consumed'])    
y = df['energy_consumed']      
plt.figure(figsize=(10,6))
plt.scatter(df['distance_km'], df['energy_consumed'], alpha=0.6)
plt.title('Distance vs Energy Consumed')    

NameError: name 'np' is not defined

### Data Description

#### The dataset for the EV Solar Project combines weather data and EV charging data to support AI-based prediction and optimization. It includes solar irradiance, cloud cover, and rainfall values, which are key factors influencing solar energy generation, along with EV charging session counts and corresponding energy consumption in kilowatt-hours. These inputs are preprocessed to handle missing values and normalized for time-series analysis. The AI model, particularly an LSTM network, uses this data to predict future solar energy output and optimize charging schedules, aiming to maximize the percentage of sessions powered by solar while reducing reliance on the grid. Evaluation metrics such as mean absolute error (MAE), solar usage percentage, and peak grid load reduction are used to assess performance, making the dataset central to demonstrating both predictive accuracy and energy management efficiency

In [ ]:
# EV Solar Project - AI Smart Charging Load Balancer
# Jupyter Notebook Code

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# ----------------------------
# 1. Load Dataset
# ----------------------------
# (You can replace this with your real dataset CSV)

try:
    df = pd.read_csv("ev_solar_project_dataset.csv")
except:
    # Generate synthetic dataset if CSV not found
    data = {
        "date": pd.date_range(start="2025-01-01", periods=100, freq="D"),
        "solar_irradiance": np.random.randint(200, 800, 100),
        "cloud_cover_percent": np.random.randint(0, 100, 100),
        "rainfall_mm": np.random.randint(0, 50, 100),
        "ev_charging_sessions": np.random.randint(5, 25, 100),
        "energy_consumed_kWh": np.random.randint(50, 200, 100)
    }
    df = pd.DataFrame(data)

print("Dataset head:")
print(df.head())

# ----------------------------
# 2. Preprocessing
# ----------------------------

df = df.fillna(method="ffill")  # handle missing values

scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[["solar_irradiance", "cloud_cover_percent", "rainfall_mm"]])

# Create time-series sequences
def create_sequences(data, seq_length=7):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length, 0])  # predict solar_irradiance
    return np.array(X), np.array(y)

seq_length = 7
X, y = create_sequences(df_scaled, seq_length)

# Train-test split
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# ----------------------------
# 3. Build LSTM Model
# ----------------------------

model = Sequential([
    LSTM(64, input_shape=(seq_length, X.shape[2])),
    Dense(1)
])

model.compile(optimizer="adam", loss="mae")
history = model.fit(X_train, y_train, epochs=20, batch_size=8, validation_split=0.2, verbose=1)

# ----------------------------
# 4. Evaluate Model
# ----------------------------

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae:.4f}")

plt.figure(figsize=(10,5))
plt.plot(y_test, label="Actual")
plt.plot(y_pred, label="Predicted")
plt.legend()
plt.title("Solar Irradiance Prediction (Normalized)")
plt.show()

# ----------------------------
# 5. Load Balancing Simulation
# ----------------------------

# Assume solar can support EV charging if irradiance > threshold
solar_threshold = 0.5
solar_usage = (y_pred.flatten() > solar_threshold).sum()
grid_usage = len(y_pred) - solar_usage

plt.bar(["Solar", "Grid"], [solar_usage, grid_usage])
plt.title("Energy Source Usage for EV Charging")
plt.show()

print("Simulation Results:")
print(f"Solar sessions: {solar_usage}")
print(f"Grid sessions: {grid_usage}")


Dataset head:
          day  solar_irradiance_kw_m2  cloud_cover_percent  rainfall_mm  \
0  2025-01-01                    3.87                 58.6          7.8   
1  2025-01-02                    6.75                 23.6          5.4   
2  2025-01-03                    5.66                 15.2         16.6   
3  2025-01-04                    4.99                 85.9          7.1   
4  2025-01-05                    2.78                 87.3          5.6   

   ev_charging_sessions  energy_consumption_kwh  
0                    46                   471.9  
1                    23                   423.2  
2                    12                   353.4  
3                    10                   448.6  
4                    14                   421.5  


C:\Users\msant\AppData\Local\Temp\ipykernel_25936\1034752212.py:39: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



KeyError: "['solar_irradiance'] not in index"